In [1]:
# %load_ext autoreload
# %autoreload 2

import os
import numpy as np

os.chdir('/home/vacekpa2/4D-RNSFP')  
import sys
# sys.path.append('../')

import torch
from data.dataloader import SFDataset4D
from pytorch3d.ops.knn import knn_points
from vis.deprecated_vis import *
from loss.flow import DT, SmoothnessLoss
from data.range_image import VisibilityScene

device = torch.device('cuda:0')

dataset = SFDataset4D(dataset_type='waymo', n_frames=5, only_first=False)
data = dataset[80]

device = torch.device('cuda:0')
pc1 = data['pc1'].to(device)
pc2 = data['pc2'].to(device)
pose12 = data['relative_pose'].to(device)

full_pc2 = data['full_pc2'][4].to(device)
id_mask1 = data['id_mask1'].to(device)

print(len(torch.unique(id_mask1)))
# Freespace = VisibilityScene(dataset='waymo', pc_scene=full_pc2)
# visibility_depth = Freespace.assign_depth_to_flow(pc1[0])
# orig_depth = pc1[0].norm(dim=1)
# valid_mask = visibility_depth > 0
# visualize_multiple_pcls(*[pc1[i].cpu() for i in range(0,3)])



112


# Freespace Notes
- The sensor is quite noisy even still
- Just accumulate the static points and refer to them when building the map
- Kabsch from map points, flow in a same way
- Use 2D raycasting from cvpr papers

# Rigid Flow and Pred Flow Notes
- NeuralPrior is bad if we calculate metric for MOS from Flow

In [11]:
# Adjacent NN for motion segmentation
from loss.flow import SmoothnessLoss, DT
import torch
from pytorch3d.ops.knn import knn_points

from data.dataloader import SFDataset4D
from pytorch3d.ops.knn import knn_points
from vis.deprecated_vis import *
from loss.flow import DT, SmoothnessLoss
from data.range_image import VisibilityScene

device = torch.device('cuda:0')

dataset = SFDataset4D(dataset_type='waymo', n_frames=5, only_first=False)
data = dataset.__getitem__(80)

pc1 = data['pc1'].to(device)
pc2 = data['pc2'].to(device)

# pc = pc1[:3]
pc = [pc1[i][pc1[i, :, 2] > 0.3] for i in range(0, 3)]

# pc = pc[:, pc[:,:,2] > 0.3]
c_pc = pc[1].unsqueeze(0)
b_pc = pc[2].unsqueeze(0)
f_pc = pc[0].unsqueeze(0)
# params
forth_flow = torch.zeros(c_pc.shape, device=device, requires_grad=True)
back_flow = torch.zeros(c_pc.shape, device=device, requires_grad=True)
optimizer = torch.optim.Adam([forth_flow, back_flow], lr=0.008)
# losses
SM_loss = SmoothnessLoss(pc1=c_pc, K=16, max_radius=1)


b_DT = DT(c_pc, b_pc)
f_DT = DT(c_pc, f_pc)

for i in range(500):
    # forth_dist, forth_nn, _ = knn_points(c_pc + forth_flow, f_pc, K=1, return_nn=True)
    # back_dist, back_nn, _ = knn_points(c_pc + back_flow, b_pc, K=1, return_nn=True)
    forth_dist, _ = f_DT.torch_bilinear_distance(c_pc + forth_flow)
    back_dist, _ = b_DT.torch_bilinear_distance(c_pc + back_flow)
    
    dist_loss = (forth_dist + back_dist).mean()
    smooth_loss = SM_loss(c_pc, forth_flow, f_pc) + SM_loss(c_pc, back_flow, f_pc)
    
    time_smooth = (forth_flow - (-back_flow)).norm(dim=2, p=1).mean()   # maybe magnitude of flow?
    
    loss = dist_loss + smooth_loss + time_smooth
    
    # print(i, loss, time_smooth)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


# DBSCAN?
from sklearn.cluster import DBSCAN
mos = (forth_flow[0].norm(dim=1, p=1) > 0.05).detach().cpu().numpy()
numpy_c_pc = c_pc.detach().cpu().numpy()
numpy_forth_flow = forth_flow.detach().cpu().numpy()

motion_pc = np.concatenate((numpy_c_pc[0, mos], numpy_forth_flow[0, mos]), axis=1)

clustering = DBSCAN(eps=0.4, min_samples=5).fit_predict(motion_pc)

ids = clustering



# visualize_points3D(motion_pc, clustering)
# VISUALS
# visualize_flow3d(c_pc[0], f_pc[0], back_flow[0])
# visualize_flow3d(c_pc[0], f_pc[0], forth_flow[0])
# visualize_points3D(c_pc[0], forth_flow[0].norm(dim=1, p=1) > 0.05)
# visualize_points3D(c_pc[0], forth_flow[0].norm(dim=1, p=1))


In [28]:
# DBSCAN?
from sklearn.cluster import DBSCAN
mos = (forth_flow[0].norm(dim=1, p=1) > 0.05).detach().cpu().numpy()
numpy_c_pc = c_pc.detach().cpu().numpy()
numpy_forth_flow = forth_flow.detach().cpu().numpy()

motion_pc = np.concatenate((numpy_c_pc[0, mos], numpy_forth_flow[0, mos]), axis=1)

clustering = DBSCAN(eps=0.4, min_samples=10).fit_predict(motion_pc)



visualize_points3D(motion_pc, clustering)

In [52]:

# 2D BEV occupancy grid
# cell_size = 0.1
# voxel_size = (cell_size, cell_size, cell_size)
# voxel_size = voxel_size
# 
# range_x, range_y, range_z = 100, 100, 10
# size = (int(range_x / voxel_size[0]), int(range_y / voxel_size[1]), int(range_z / voxel_size[2]))
# mid = torch.tensor([size[0] / 2, size[1] / 2, size[2] / 2], device=device, dtype=torch.long)
# bev_grid = - torch.ones(size[:2], device=device, dtype=torch.long)
# bev_grid.shape

# flow urcuje dyn/static

# visualize_multiple_pcls(*[pc2[0].cpu(), pc1[0].cpu()])